In [ ]:
# coding: utf-8
import numpy as np
from matplotlib.pyplot import subplots, show
from kapteyn import kmpfit

In [ ]:
data = np.load("chisqs.B0329+54.npz")
period = data['period']
chisq = data['chisq']

In [ ]:
def gauss(p,x):
    return p[0]*np.exp(-(x-p[2])**2/p[1]**2)+p[3]

def absexp(p,x):
    return p[0]*np.exp(-np.abs(x-p[2])/p[1])+p[3]

def lorentz(p,x):
    return p[0]*p[1]**2/((x-p[2])**2 + p[1]**2)+p[3]

model = absexp

fitobj=kmpfit.simplefit(model,[6e-10, 1e-6, period.mean()-1e-5, 0.4e-10], period, chisq)

In [ ]:
print("\nFit status kmpfit:")
print("====================")
print("Best-fit parameters:        ", fitobj.params)
print("Asymptotic error:           ", fitobj.xerror)
print("Error assuming red.chi^2=1: ", fitobj.stderr)
print("Chi^2 min:                  ", fitobj.chi2_min)
print("Reduced Chi^2:              ", fitobj.rchi2_min)
print("Iterations:                 ", fitobj.niter)
print("Number of free pars.:       ", fitobj.nfree)
print("Degrees of freedom:         ", fitobj.dof)

In [ ]:
from scipy.interpolate import interp1d
from scipy.optimize import differential_evolution

In [ ]:
interp = interp1d(period,chisq,kind='cubic')
bestperiod = differential_evolution(lambda x: -interp(x), np.array([period.min(),period.max()])[None,:])

In [ ]:
bestperiod.x - period.mean()

In [ ]:
fig, ax = subplots()
ax.plot(period,chisq)
ax.plot(period, model(fitobj.params, period))
ax.axvline(period.mean())
ax.axvline(fitobj.params[2])
ax.axvline(bestperiod.x)
ax.set_xlim(fitobj.params[2]-1e-6, fitobj.params[2]+1e-6)
#ax.set_ylim(1.2e-7, 1.35e-7)
#ax.set_xlim(56.5,57)
show()

In [ ]:
print(period.mean()-fitobj.params[2])

In [ ]:
from observation import Observation
obs = Observation("data/obs-10-04-2018/B2111+46_10-04-2018.fits.gz")

In [ ]:
from astropy.io import fits
import numpy as np

hdulist = fits.open("data/obs-10-04-2018/B2310+42-04-2018.fits.gz")
col_period = fits.Column(name='period', array=data['period'], format='D')
col_DM = fits.Column(name='DM', array=data['DM'], format='D')
col_chisq = fits.Column(name='chisq', array=data['chisq'], format='D')
t = fits.BinTableHDU.from_columns([col_period, col_DM, col_chisq])
t.header['fitpars'] = str(fitobj.params)
t.header['bestp'] = fitobj.params[2]
t.header['fitfunc'] = model.__name__
t.header['fitpars'] = "-"
t.header['bestp'] = bestperiod.x[0]
t.header['fitfunc'] = 'maximum'
hdulist.append(t)
#hdulist.writeto("data/obs-10-04-2018/B2310+42-04-2018-withP.fits")